# Production Problem - manage the production of bakery products to meet demand with your resources.

The model aims at minimizing the production cost for a number of products while satisfying customer demand.

   * Each product can be produced either inside the company(in-house) or outside(outsourced) at a higher cost.
   * The in-house production is constrained by the company's resources but for the outsourced production there is no resource constraint

The model first declares the products and the resources.
The data consists of the description of the products (the demand, the in-house and outsourced costs,
and the resource consumption) and the in-house capacity of the various resources.
The variables for this problem are the in-house and outsourced production for each product.

In [37]:
# Import the libraries
import docplex.mp

In [38]:
# Model the data
# description of the products (product-name, the demand, in-house cost, outsourced cost)
# the in-house capacity of the various resources 
# the resource consumptions for each product
products = [("BlackForestCake", 100, 0.6, 0.8),
            ("RedVelvetCake", 200, 0.8, 0.9),
            ("ButterCookies", 300, 0.3, 0.4)]

# resources are a list of tuples (name, capacity)
in_house_resource_capacity = [("flour", 20),
             ("eggs", 40)]

consumptions = {("BlackForestCake", "flour"): 0.5,
                ("BlackForestCake", "eggs"): 0.2,
                ("RedVelvetCake", "flour"): 0.4,
                ("RedVelvetCake", "eggs"): 0.4,
                ("ButterCookies", "flour"): 0.3,
                ("ButterCookies", "eggs"): 0.6}

In [39]:
# Create the DOcplex model - use CPLEX Modeling for Python to build a Mixed Integer Programming (MIP) model for this problem.
# The model contains all the business constraints and defines the objective.

from docplex.mp.model import Model
mdl = Model(name="Bakery")

In [40]:
# Define the decision variables
inHouse = mdl.continuous_var_dict(products, name='in-house')
outsourced = mdl.continuous_var_dict(products, name='outsourced')

In [41]:
# Express the business constraints
# 1. Each product can be produced either inside the company or outside, at a higher cost.
# 2. The inside production is constrained by the company's resources, while outside production is considered unlimited.

# demand satisfaction
mdl.add_constraints((inHouse[prod] + outsourced[prod] >= prod[1], 'ct_demand_%s' % prod[0]) for prod in products)

# --- resource capacity ---
mdl.add_constraints((mdl.sum(inHouse[p] * consumptions[p[0], res[0]] for p in products) <= res[1], 'ct_res_%s' % res[0]) for res in in_house_resource_capacity)

mdl.print_information()

Model: Bakery
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: none
 - problem type is: LP


In [42]:
# Express the objective
# Minimizing the production cost for a number of products while satisfying customer demand
total_inHouse_cost = mdl.sum(inHouse[p] * p[2] for p in products)
total_outsourced_cost = mdl.sum(outsourced[p] * p[3] for p in products)

mdl.minimize(total_inHouse_cost + total_outsourced_cost)

In [43]:
# Solve with Decision Optimization
mdl.solve()

docplex.mp.solution.SolveSolution(obj=372,values={in-house_BlackForestCa..

In [44]:
# Investigate the solution and then run an example analysis
obj = mdl.objective_value

print("* Production model solved with objective: {:g}".format(obj))
print("* Total inside cost=%g" % total_inHouse_cost.solution_value)
for p in products:
    print("Inside production of {product}: {ins_var}".format(product=p[0], ins_var=inHouse[p].solution_value))
print("* Total outside cost=%g" % total_outsourced_cost.solution_value)
for p in products:
    print("Outside production of {product}: {out_var}".format(product=p[0], out_var=outsourced[p].solution_value))

* Production model solved with objective: 372
* Total inside cost=24
Inside production of BlackForestCake: 40.0
Inside production of RedVelvetCake: 0
Inside production of ButterCookies: 0
* Total outside cost=348
Outside production of BlackForestCake: 60.0
Outside production of RedVelvetCake: 200.0
Outside production of ButterCookies: 300.0


In [45]:
mdl.export_as_lp(path='802-Production Problem.lp')

'802-Production Problem.lp'

In [46]:
mdl.print_information()

Model: Bakery
 - number of variables: 6
   - binary=0, integer=0, continuous=6
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: minimize
 - problem type is: LP
